binary only dataframe

classes and functions

In [21]:
class patient:
    def __init__(self, filepath, input_fileName, lengthPtID):
        ptID = input_fileName[:lengthPtID]
        self.ptID = ptID
        self.fileName = input_fileName
        #find the demographics in one of two sheets
        # directory = os.getcwd()
        file_adds = filepath + '/' + input_fileName
        file = pd.ExcelFile(file_adds)
        sheet_Names = file.sheet_names
        if 'Cancer_Registery Data' in sheet_Names:
            #do the thing for getting patient info
            df = pd.read_excel(input_fileName, sheet_name= 'Cancer_Registery Data')
            self.ptDemographics = df
            self.amyloid_status = df.at[0,"Amyloid Status"]
            self.survival = df.at[0,"Survival Time (Months)"]
            self.ageDx = df.at[0,"Age At Diagnosis"] 
            self.sex = df.at[0,"Gender Cancer Registry"]
            self.vitalStatus = df.at[0, "Vital Status Cancer Registry"]
            self.dx = df.at[0, "Histology"]

        elif '30 Cerner Patients ' in sheet_Names:
            #do the thing for getting patient info
            df = pd.read_excel(input_fileName, sheet_name= '30 Cerner Patients ')
            self.ptDemographics = df
            self.amyloid_status = df.at[0,"Amyloid Status"]
            self.survival = df.at[0,"Survival Time (months)"]
            self.ageDx = df.at[0,"Age At Diagnosis"] 
            self.sex = df.at[0,"Gender Cerner"]
            self.vitalStatus = df.at[0, "Vital Status (Cerner)"]
            self.dx = df.at[0, "Primary Site"]


        else: print("no patient identifying information")

        #now iterate through the list of sheets to import the patient data 
        if 'Labs' in sheet_Names:
            self.labsData = pd.read_excel(input_fileName, sheet_name= 'Labs')
        else: self.labsData = False
        
        if 'amyloid in subject' in sheet_Names:
            self.echoData = pd.read_excel(input_fileName, sheet_name= 'amyloid in subject')
        else: self.echoData = False

        if 'ChemoTx' in sheet_Names:
            self.ChemoTx = pd.read_excel(input_fileName, sheet_name= 'ChemoTx')
        else: self.ChemoTx = False

        if 'Hematologic Tx' in sheet_Names:
            self.HemeTx = pd.read_excel(input_fileName, sheet_name= 'Hematologic Tx')
        else: self.HemeTx = False

        if 'Immuno Tx' in sheet_Names:
            self.ImmunoTx = pd.read_excel(input_fileName, sheet_name= 'Immuno Tx')
        else: self.ImmunoTx = False

        if 'Radiation Tx' in sheet_Names:
            self.RadTx = pd.read_excel(input_fileName, sheet_name= 'Radiation Tx')
        else: self.RadTx = False

        if 'Other Tx' in sheet_Names:
            self.OtherTx = pd.read_excel(input_fileName, sheet_name = 'Other Tx')
        else: self.OtherTx = False

        if 'Physican_Notes' in sheet_Names:
            self.MDnotes = pd.read_excel(input_fileName, sheet_name= 'Physican_Notes')
        else: self.MDnotes = False

        #to create a callable list of treatments recieved for which we have data
        tx_sheets = ['ChemoTx', 'Hematologic Tx', 'Immuno Tx']
        treatmentTypesRecieved = []

        for possible in tx_sheets:
        
            if possible in sheet_Names:
                treatmentTypesRecieved.append(possible)
        self.txRecieved = treatmentTypesRecieved

def FilterList(list, keyWords_primary, keyWords_secondary = ["Empty"], omit = ["Empty"]):
    #store the filtered result
    filteredList = []

    if keyWords_secondary == ["Empty"]:
        for c in list: #loop the columns
            for buzz in keyWords_primary: #loop the key words
                if buzz in c: #if the column contains the key word
                    filteredList.append(c) #add the column to the list
                    break #do not continue testing primary key words for this column
    
    else: 
        for c in list: #loop the columns
            for buzz in keyWords_primary: #loop the key words
                if buzz in c: #if the column contains the key word
                    for secondary in keyWords_secondary:
                        if secondary in c:
                            filteredList.append(c) #add the column to the list
                            break #has been added to list based on passing the seondary 
                        else:
                            pass
                    break #do not continue to check primary key words for this column
                else:
                    pass
                
    if omit != ["Empty"]: 
        for x in omit: 
            for a in filteredList:
                if x in a:
                    filteredList.remove(a)
    
    #by iterating through the column names first, we keep the order of the columns

    return filteredList 

#createBinary function
def createBinary(df, listOfLabs, timeColumn): 
    #list of labs needs to match exactly.  
    # Filter the dataframe before inputting
    #for boolean
    df_toBool = df[listOfLabs]
    df_bool = df_toBool.notna()
    df_asint = df_bool.astype(int)
    #add column back for time 
    df_asint.insert(0, timeColumn, df[timeColumn])
    return df_asint

def createBinarySum(df, listOfLabs, timeColumn, defaxis):
    #list of labs needs to match exactly.  
    # Filter the dataframe before inputting
    #for boolean
    df_toBool = df[listOfLabs]
    df_bool = df_toBool.notna()
    df_asint = df_bool.astype(int)
    #add column back for time 
    # df_asint.insert(0, timeColumn, df[timeColumn])
    #sum across the row
    #defaxis = 0 means the columns are preserved 
    #defaxis =1 means the rows are preserved
    df_new_sum = df_asint.sum(axis = defaxis)
    return df_new_sum

#get list of patients from directory 
def GetListOfPatientsFromDirectory (directory, fileExtension, fileCommon, lengthID):
    #get the directory 
    listFileNames = os.listdir(directory)
    outputList = []
    #loop through list of fileNames: 
    for file in listFileNames:
        #id if the file is patient data
        if fileCommon in file and file.endswith(fileExtension):
            #trim the file name to the pt ID
            ptID = file[:lengthID]
            outputList.append(ptID)
    return outputList


def GetListofPTfiles(directory, fileExtension, fileCommon):
    listFileNames = os.listdir(directory)
    outListFiles = []
    #loop through file names
    for file in listFileNames:
        if fileCommon in file and file.endswith(fileExtension):
            outListFiles.append(file)
    return outListFiles
def getNormalValue(filteredDataColumn, skipValue = 6666):
    test = filteredDataColumn
    #input is a series
    
    for entry in test:
        if entry != skipValue:
            outputValue = entry
                # print("this is output: ", outputValue)
                #set x to True
            break
        else:
            # print("entry is blank")
            pass
    return outputValue

#get amyloid status dictionary inputs, sort ptIDs to lists based on amyloid status 
def getListSortedByAmyloid(amyloidDictionary, listPositive, listNegative, listOther,
                           statusPositive = "yes", statusNegative = "no"):
    for pt in amyloidDictionary.keys():
        status = amyloidDictionary[pt]
        if status == statusPositive:
            listPositive.append(pt)
        elif status == statusNegative:
            listNegative.append(pt)
        else:
            listOther.append(pt)
    #does not check for double listed patient IDs
    
#function for masking a dataframe 
def patientSectionOfFrame(start_frame, column_toMask, ptID):
    #ptID is a row parameter in the column_toMask
    df_use = start_frame #copy of the frame

    #first check if sheet has the column of interest
    data_top = list(start_frame.columns) #returns the headers as a list

    #determine of the sheet contains the column of interest
    if data_top.count(column_toMask) > 0: 
        #if true, now find if the patient is present
        searchForPt = start_frame[column_toMask].str.startswith(ptID).sum() #returns the number of instances of the patient
        if searchForPt > 0:
            #patient is present 
            #mask the data for the patient
            df_clean = df_use.dropna(subset= [column_toMask]) #removing NA 
            mask = df_clean[column_toMask].str.startswith(ptID) #masking for the rows where the patient has data
            df_sub_tosave = df_clean[mask] #output dataframe filtered for the patient
            return df_sub_tosave
        else:
            #the patient isn't present
            # return print("The patient " + ptID + " is not in " + sheet_name)
            pass
    else:
        #the column of interest isn't present
        # return print("The column " + column_toMask + " is not in the sheet " + sheet_name)
        pass


def outputToExcel(df_data, fileName_header, fileName_Common, fileName_suffix,
                  parent_dir, folderName, sheetName):
    fileName = fileName_header + fileName_Common + fileName_suffix
    outfile_extension = '.xlsx'
    outfile_boxplts = fileName + outfile_extension
    path_out= os.path.join(parent_dir, folderName, outfile_boxplts)

    if os.path.exists(path_out):
            #if old sheet
            with pd.ExcelWriter(path_out, mode = 'a', if_sheet_exists = 'overlay') as writer:
                    df_data.to_excel(writer, sheet_name = sheetName, index = True)
    else: 
            #new sheet
            with pd.ExcelWriter(path_out) as writer:
                    df_data.to_excel(writer, sheet_name = sheetName, index = True) #if new sheet
    
def outputFiguresPath(fileName_header, fileName_mid, fileName_suff, parent_dir, folderName):
        fileName_header = str(fileName_header)
        fileName_mid = str(fileName_mid)
        fileName_suff = str(fileName_suff)

        #cleaning the input to prevent addition of / to the directory
        fileName_header = fileName_header.replace("/", "-")
        fileName_mid = fileName_mid.replace("/", "-")
        fileName_suff = fileName_suff.replace("/", "-")

        fileName = fileName_header + fileName_mid +fileName_suff
        out_filename = fileName + '.tif'
        new_filepath = os.path.join(parent_dir, folderName, out_filename)
        #the output path can be used to save the figure
        return new_filepath
        # plt.savefig(new_filepath, bbox_inches = 'tight')

def makeFolderPathForData(parent_dir, folderName_header, folderName_common, folderName_suffix):
    #makes a new directory for your files
    #returns the folder name for use in other functions
    folderName = folderName_header + folderName_common + folderName_suffix
    path = os.path.join(parent_dir,folderName)
    if not os.path.exists(path):
        os.mkdir(path)
    return folderName

In [22]:
#libraries and dirrectory 
import pandas as pd
import numpy as np
import os
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from scipy.optimize import curve_fit

In [23]:
#location of original data
starting_directory = os.getcwd()

new_dirrectory = 'C:/Users/maega/Documents/3000 PhD/3300_BEAT Labs/Projects/Cardiac-Amyloidosis-Multiple-Myeloma/012 Processed Data/data-2024-06-05'
os.chdir(new_dirrectory)

In [24]:
#file name suffix 
outputSheet_Suffix = "_2024-07-10_v1"
outputSheet_common = "-LabsBinary-wTx-Summary"

In [25]:
#output dirrectory
outputDir = "C:/Users/maega/Documents/3000 PhD/3300_BEAT Labs/Projects/Cardiac-Amyloidosis-Multiple-Myeloma/012 Processed Data"
folderNameOut = makeFolderPathForData(parent_dir=outputDir, folderName_header="DFsForSVM", folderName_common="_2024-07-10", folderName_suffix="_v1")

In [26]:
#filler information for fits of 1 point
one_point_STD = 0
one_point_r2 = 1000000
one_point_residual_error = 1000000

importing patient data

In [27]:
#importing patient data as dictionaries of dataframes

#build a dictionary of patients and their values 
ptFiles = GetListofPTfiles(new_dirrectory, '.xlsx', 'data')

patientDictAmyloid = {}
patientDictSurvival = {}
patientDictLabs = {}
patientDictEcho = {}
patientDictNotes = {}
patientDictDx = {}
patientDictVitalStatus = {}
patientDictSex = {}
patientDictAge = {}
patientDictChemo = {}
patientDictImmuno = {}
patientDictHeme = {}
patientDictRad = {}
patientDictOther = {}

#loop through the listOfPatients 
for file in ptFiles: 
    pt = patient(new_dirrectory, file, 4)
    patientDictAmyloid[pt.ptID] = pt.amyloid_status
    patientDictSurvival[pt.ptID] = pt.survival
    patientDictLabs[pt.ptID] = pt.labsData
    patientDictEcho[pt.ptID] = pt.echoData
    patientDictNotes[pt.ptID] = pt.MDnotes
    patientDictDx[pt.ptID] = pt.dx
    patientDictVitalStatus[pt.ptID] = pt.vitalStatus
    patientDictSex[pt.ptID] = pt.sex
    patientDictAge[pt.ptID] = pt.ageDx
    patientDictChemo[pt.ptID] = pt.ChemoTx
    patientDictImmuno[pt.ptID] = pt.ImmunoTx
    patientDictHeme[pt.ptID] = pt.HemeTx
    patientDictRad[pt.ptID] = pt.RadTx
    patientDictOther[pt.ptID] = pt.OtherTx

patients by amyloid status

In [28]:
#list of patients by amyloid status 

listPositive = []
listNegative = []
listOther = []

getListSortedByAmyloid(patientDictAmyloid, listPositive= listPositive,
                       listNegative=listNegative, listOther=listOther)

#first make a copy of the list
listALKnown = listPositive.copy()
#add the negative patients to the list of positive patients
listALKnown.extend(listNegative) 

In [29]:
#length of groups 
print("Lenght of positive: ", len(listPositive))
print("length of negative: ", len(listNegative))
print("list of unknown: ", len(listOther))

Lenght of positive:  24
length of negative:  24
list of unknown:  19


collecting patient demographics

In [30]:
dict_dx_codes = {
    "MGUS" : 1, 
    "MM" : 2, 
    "Multiple Myeloma" : 2,
    "MULTIPLE MYELOMA" : 2 
    #other values are going to be set to 0
}

dict_AL_codes = {
    "yes" : 2, 
    "no" : 1, 
    "unk" : 3,
}

dict_sex_codes = {
    "Male" : 0,
    "MALE": 0, 
    "Female" : 1, 
    "FEMALE" : 1,  
}

dict_of_codes = {"AL": dict_AL_codes, 
                 "Dx": dict_dx_codes, 
                 "Sex": dict_sex_codes}

dict_of_data = {"AL": patientDictAmyloid, 
                "Dx": patientDictDx, 
                "Sex": patientDictSex}

dict_encoded_pt_AL_Dx_sex = {}

for ptID in list(patientDictDx.keys()):
    listPtValues = []
    for key in list(dict_of_codes.keys()):
        LocalValue = ''
        for code in dict_of_codes[key]:
            if code in dict_of_data[key][ptID]:
                # print(ptID, " code: ", code)
                LocalValue = dict_of_codes[key][code] 
        if LocalValue == '': 
            listPtValues.append(0)
        else:
             listPtValues.append(LocalValue)
    # print("patient values: ", listPtValues)
    dict_encoded_pt_AL_Dx_sex[ptID] = listPtValues

df_all_encoded_AL_dx_sex = pd.DataFrame.from_dict(dict_encoded_pt_AL_Dx_sex, orient='index', columns=["Amyloid Status", "Dx", "Sex"])

# df_all_encoded_AL_dx_sex.columns = ["Amyloid Status", "Dx", "Sex"]

In [31]:
#getting age and survival 
df_all_ages = pd.DataFrame.from_dict(patientDictAge, orient= 'index', columns= ['Age'])
df_all_survival = pd.DataFrame.from_dict(patientDictSurvival, orient='index', columns= ['Survival (in months)'])

#combining for demographics

df_all_demographics = pd.concat([df_all_encoded_AL_dx_sex, df_all_ages, df_all_survival], axis= 1)

#slicing for amyloid known and unknown

df_ALKnown_demographics = df_all_demographics.loc[listALKnown]
df_ALOther_demographics = df_all_demographics.loc[listOther]

outputToExcel(df_data = df_ALKnown_demographics, fileName_header= "AL-KnownPts", fileName_Common= outputSheet_common, fileName_suffix= outputSheet_Suffix,
            parent_dir= outputDir, folderName=folderNameOut, sheetName= "demographics")
outputToExcel(df_data = df_ALOther_demographics, fileName_header= "AL-UnknownPts", fileName_Common= outputSheet_common, fileName_suffix= outputSheet_Suffix,
            parent_dir= outputDir, folderName=folderNameOut, sheetName= "demographics")

collecting patient treatment information

In [32]:
#collecting information about treatments patient has recieved

#each condition appends 2 numbers
#first number is a boolean if the patient has data
#the second number is the total number of treatments the patient has had.

#list of dictionaries 
pt_Tx_dict_to_iterate = [patientDictChemo, patientDictImmuno, patientDictHeme, patientDictRad, patientDictOther]

dictPt_Tx_Summary = {}

for ptID in list(patientDictAge.keys()): 
    #get the treatment information for that patient
    ptTxList = [] 
    for tx in pt_Tx_dict_to_iterate:
        if type(tx[ptID]) != bool: 
            ptTxList.append(1)
            #get the length of the data
            numb_txs = len(tx[ptID])
            ptTxList.append(numb_txs)
        else:
            ptTxList.append(0)
            ptTxList.append(0)

    dictPt_Tx_Summary[ptID] = ptTxList

df_all_tx_summary = pd.DataFrame.from_dict(dictPt_Tx_Summary, orient='index')
df_all_tx_summary.columns = ["ChemoTx", "ChemoTx Number", "ImmunoTx", "ImmunoTx Number", "HemeTx", "HemeTx Number", 
"RadTx", "RadTx Number", "OtherTx", "OtherTx Number"]

In [33]:
#splitting dataframe by amyloid status
df_AL_known_tx_summary = df_all_tx_summary.loc[listALKnown]
df_AL_Unknown_tx_summary = df_all_tx_summary.loc[listOther]
outputToExcel(df_data = df_AL_known_tx_summary, fileName_header= "AL-KnownPts", fileName_Common= outputSheet_common, fileName_suffix= outputSheet_Suffix,
            parent_dir= outputDir, folderName=folderNameOut, sheetName= "txSummary")
outputToExcel(df_data = df_AL_Unknown_tx_summary, fileName_header= "AL-UnknownPts", fileName_Common= outputSheet_common, fileName_suffix= outputSheet_Suffix,
            parent_dir= outputDir, folderName=folderNameOut, sheetName= "txSummary")

collecting physician note information


In [34]:
#dictionaries for conversion
dictECOGtoKPS = {0 : 95,
                 1 : 75, 
                 2 : 55, 
                 3 : 35, 
                 4 : 15, 
                 5 : 0}

dictKPStoECOG = {100: 0, 90: 0, 
                 80: 1, 70: 1, 
                 60: 2, 50: 2, 
                 40: 3 , 30: 3, 
                 20: 4, 10: 4, 0: 5 }

dictClinicalImpression = {
    "Complete Response/No Evidence of Disease": 4,
    "Stable Disease": 1,
    "Partial Response": 3, 
    "Progressive Disease": 2, 
    "Not Applicable": 0,
    "Not Assessable": 0}
#this preserves NAN values 

#create new dictionary for storing the scored notes information
dictMDnotesScored = {}

#call each patient in the dictionary storing patient note information
for ptID in list(patientDictNotes.keys()):
    #check to make sure the patient has note data
    if not isinstance(patientDictNotes[ptID], bool):
        #get the patient notes
        ptNotes = patientDictNotes[ptID]

        #column names of interest
        listColumnNames = ['time from diagnosis to note (months)', 'ECOG Performance Status', 'Karnofsky Performance Status', 'Clinical Impression of Disease']
        ptNotesTrimmed = ptNotes[listColumnNames]
        
        #convert the Karnofsky scores to integers
        ptNotesTrimmed['Karnofsky Performance Status'] = ptNotesTrimmed['Karnofsky Performance Status'].astype(str).str.extract('(\d+)').astype(float).astype('Int64')


        for i in range(len(ptNotesTrimmed['time from diagnosis to note (months)'])):
            #get the scores for that row
            ptTime = ptNotesTrimmed.at[i, 'time from diagnosis to note (months)']
            ptECOG = ptNotesTrimmed.at[i,'ECOG Performance Status']
            ptKPS = ptNotesTrimmed.at[i, 'Karnofsky Performance Status']
            ptClinicalImpression = ptNotesTrimmed.at[i, 'Clinical Impression of Disease']

            #if the value of ptECOG was "OTHER", then the row should be dropped.
            if ptECOG == "OTHER":
                ptNotesTrimmed.drop(index = i, inplace=True )
                #also drops full row, so if there is clinical information add time back

            #infer ECOG from Karnofsky where you now have half points
            elif np.isnan(ptECOG) ==True and pd.isna(ptKPS) == False: 
                ptECOG = dictKPStoECOG[ptKPS]

            #infer Karnofsky from ECOG 
            elif pd.isna(ptKPS) == True and np.isnan(ptECOG) ==False: 
                ptKPS = dictECOGtoKPS[ptECOG]

            #save these values back to the dataframe
            if ptECOG != "OTHER":
                ptNotesTrimmed.at[i,'ECOG Performance Status'] = ptECOG
                ptNotesTrimmed.at[i, 'Karnofsky Performance Status'] = ptKPS

            #checking to see if the patient has a clinical impression input
            #if there is a string, convert to a numeric value using the dictionary
            if type(ptClinicalImpression) == str:
                ptClinicalImpressionNumeric = dictClinicalImpression[ptClinicalImpression]
                ptNotesTrimmed.at[i, 'Clinical Impression of Disease'] = ptClinicalImpressionNumeric
                ptNotesTrimmed.at[i, 'time from diagnosis to note (months)'] =ptTime
                #you can nest this under the =! "Other" if you find the rows with no ECOG/Karnofsky data to not be helpful
                            
        #saving the notes to the new dictionary
        dictMDnotesScored[ptID] = ptNotesTrimmed


C:\Users\maega\AppData\Local\Temp\ipykernel_31568\1591119509.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ptNotesTrimmed['Karnofsky Performance Status'] = ptNotesTrimmed['Karnofsky Performance Status'].astype(str).str.extract('(\d+)').astype(float).astype('Int64')
C:\Users\maega\AppData\Local\Temp\ipykernel_31568\1591119509.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ptNotesTrimmed['Karnofsky Performance Status'] = ptNotesTrimmed['Karnofsky Performance Status'].astype(str).str.extract('(\d+

In [35]:
#now to make this a set of descriptive statistics that can be used in the dataframe
#challenged by nan values
descriptive_stats_dict_notes = {}

for pt in list(dictMDnotesScored.keys()):
    ptNoteData = dictMDnotesScored[pt]

    #create structure for one patient 
    columnTracker = 0
    # LabDescStatsSeries = pd.Series()
    listColumnNames_woTime = ['ECOG Performance Status', 'Karnofsky Performance Status', 'Clinical Impression of Disease']

    for col in listColumnNames_woTime:
        ptColData = ptNoteData[col]
        # ptLabData = ptLabData.dropna()

        #descriptive stats
        ColDescStats = ptColData.dropna().astype(int).describe()
        orig_indices = ColDescStats.index
        newIndices = [col + " Desc " + str(idx) for idx in orig_indices]
        ColDescStats.index = newIndices
        
        #check to see if STD is not a number... or if there is only 1 data point 
        colCounts = col + " Desc " + "count" 
        colSTD = col + " Desc " + "std" 
        if ColDescStats[colCounts] == 1: 
            ColDescStats[colSTD] = one_point_STD

        #add the data to a constant list

        if columnTracker == 0: 
            #if this is a new patient and the first lab
            #create the series variable
            ColDescStatsSeries = ColDescStats.copy()
        if columnTracker > 0: 
            #same patient as last patient, new lab
            #add the data to the series
            ColDescStatsSeries = ColDescStatsSeries._append(ColDescStats)

        #increment to track if the data is for a new patient or not
        columnTracker = columnTracker + 1
    #saving the data from the patient to a dictionary
    descriptive_stats_dict_notes[pt] = ColDescStatsSeries

#converting the dictionary to a dataframe
#rows are patient IDs
#columns are the descriptive statistics for a particular lab
df_all_DescriptiveStats_notes = pd.DataFrame.from_dict(descriptive_stats_dict_notes, orient='index')

In [36]:
#binarization of physician note information:
#dictionaries for conversion
dictECOGtoKPS = {0 : 95,
                 1 : 75, 
                 2 : 55, 
                 3 : 35, 
                 4 : 15, 
                 5 : 0}

dictKPStoECOG = {100: 0, 90: 0, 
                 80: 1, 70: 1, 
                 60: 2, 50: 2, 
                 40: 3 , 30: 3, 
                 20: 4, 10: 4, 0: 5 }

dictClinicalImpression = {
    "Complete Response/No Evidence of Disease": 4,
    "Stable Disease": 1,
    "Partial Response": 3, 
    "Progressive Disease": 2, 
    "Not Applicable": 0,
    "Not Assessable": 0}
#this preserves NAN values 

#create new dictionary for storing the scored notes information
dictMDnotesScored = {}

#call each patient in the dictionary storing patient note information
for ptID in list(patientDictNotes.keys()):
    #check to make sure the patient has note data
    if not isinstance(patientDictNotes[ptID], bool):
        #get the patient notes
        ptNotes = patientDictNotes[ptID]

        #column names of interest
        listColumnNames = ['time from diagnosis to note (months)', 'ECOG Performance Status', 'Karnofsky Performance Status', 'Clinical Impression of Disease']
        ptNotesTrimmed = ptNotes[listColumnNames]
        
        #convert the Karnofsky scores to integers
        ptNotesTrimmed['Karnofsky Performance Status'] = ptNotesTrimmed['Karnofsky Performance Status'].astype(str).str.extract('(\d+)').astype(float).astype('Int64')
        
        #saving the notes to the new dictionary
        dictMDnotesScored[ptID] = ptNotesTrimmed

dict_bool_notes_collected = {}

dict_numb_notes_collected = {}

x_column_name_notes = "time from diagnosis to note (months)"

for ptID in list(patientDictNotes.keys()):
    #get column names
    columnNames_notes = ['ECOG Performance Status', 'Karnofsky Performance Status', 'Clinical Impression of Disease']

    if not isinstance(patientDictNotes[ptID], bool):
        #get the patient data 
        ptNoteData = dictMDnotesScored[ptID]
        #get number of labs collected 
        lengthOfData = len(ptNoteData[x_column_name_notes])

        #get column names
        # columnNames_notes = listColumnNames = ['ECOG Performance Status', 'Karnofsky Performance Status', 'Clinical Impression of Disease']

        if lengthOfData > 1: 
            numb_data_collected = createBinarySum(ptNoteData, columnNames_notes, x_column_name_notes, defaxis= 0)
            bool_data_collected = numb_data_collected.copy()
            for lab in columnNames_notes:
                if numb_data_collected[lab] > 0: 
                    #data was collected return 1
                    bool_data_collected[lab]= 1
                elif numb_data_collected[lab] == 0: 
                    #no data collected, return 0
                    bool_data_collected[lab] = 0

        elif lengthOfData == 1: 
            numb_data_collected = createBinary(ptNoteData, columnNames_notes, x_column_name_notes)
            numb_data_collected= numb_data_collected.drop(columns = x_column_name_notes) #remove time column
            numb_data_collected= numb_data_collected.squeeze() #convert to series
            bool_data_collected = numb_data_collected.copy()
    
    elif isinstance(patientDictNotes[ptID], bool): 
        numb_data_collected = pd.Series(data =[0,0,0], index = columnNames_notes)
        bool_data_collected = pd.Series(data = [0,0,0], index = columnNames_notes)

    #renaming the number recorded columns columns 
    orig_columnNames = numb_data_collected.index
    new_columnNames = [str(idx) + " Number of Points " for idx in orig_columnNames]
    numb_data_collected.index = new_columnNames

    #renaming columns for boolean
    orig_columnNames_bool = bool_data_collected.index
    new_columnNames_bool = [str(idx) + " binary "  for idx in orig_columnNames_bool]
    bool_data_collected.index = new_columnNames_bool

    dict_bool_notes_collected[ptID] = bool_data_collected
    dict_numb_notes_collected[ptID] = numb_data_collected

#convert to dataframes
df_all_boolNotes = pd.DataFrame.from_dict(dict_bool_notes_collected, orient= 'index')
df_all_numbNotes = pd.DataFrame.from_dict(dict_numb_notes_collected, orient='index')

#concatinate 
df_all_notes_bin_count = pd.concat([df_all_boolNotes, df_all_numbNotes], axis= 1)

C:\Users\maega\AppData\Local\Temp\ipykernel_31568\2208754726.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ptNotesTrimmed['Karnofsky Performance Status'] = ptNotesTrimmed['Karnofsky Performance Status'].astype(str).str.extract('(\d+)').astype(float).astype('Int64')
C:\Users\maega\AppData\Local\Temp\ipykernel_31568\2208754726.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ptNotesTrimmed['Karnofsky Performance Status'] = ptNotesTrimmed['Karnofsky Performance Status'].astype(str).str.extract('(\d+

In [37]:
#splitting dataframe by amyloid status
df_AL_known_notes_bin_count = df_all_notes_bin_count.loc[listALKnown]
df_AL_Unknown_notes_bin_count = df_all_notes_bin_count.loc[listOther]
outputToExcel(df_data = df_AL_known_notes_bin_count, fileName_header= "AL-KnownPts", fileName_Common= outputSheet_common, fileName_suffix= outputSheet_Suffix,
            parent_dir= outputDir, folderName=folderNameOut, sheetName= " notes")
outputToExcel(df_data = df_AL_Unknown_notes_bin_count, fileName_header= "AL-UnknownPts", fileName_Common= outputSheet_common, fileName_suffix= outputSheet_Suffix,
            parent_dir= outputDir, folderName=folderNameOut, sheetName= "notes")

collecting patient labs

In [38]:
#dictionary for the filtered data and the list of columnNames
#this enables filtering of the patient data as its own block of code
dictOfFilteredLabs = {}

#other input values to set outside of loop 
x_column_name = "RelTime(Days)"
#creating dictionaries for all patients in the dirrectory 

#also collects the column names of interest 

for ptID in (patientDictLabs.keys()):
    #to collect and filter the patient labs into a dictionary
    df_Labs = patientDictLabs[ptID]

    orig_columns = df_Labs.columns

    #filter the patient labs
    columnNames = FilterList(orig_columns, keyWords_primary = ['Result', 'Troponin-T', 'BNP'],  
                                omit = ['Method', 'ABO', 'Dohle', "Toxic Granulation", "D Dimer Result",
                                        "Urine Color Result", "Urine Leukocyte Esterase Result",
                                        "Urine Nitrite Result", "Urinalysis Squamous Epithelial Cells Result (/hpf)",
                                        "(/hpf)", "Hep B", "Hep B Surf Ag Result", "Hep C", "Hep C Ab Screen Result",
                                        "Hypersegmentation Result", "HCG Result (Serum)", "Coombs Test Result",
                                        "HIV Result"])
    # columnNames = FilterList(orig_columns, keyWords_primary = columnsToUse,
    #                          omit = ['Method', 'ABO', 'Dohle', "Toxic Granulation", "D Dimer Result",
    #                                     "Urine Color Result", "Urine Leukocyte Esterase Result",
    #                                     "Urine Nitrite Result", "Urinalysis Squamous Epithelial Cells Result (/hpf)",
    #                                     "(/hpf)", "Hep B", "Hep B Surf Ag Result", "Hep C", "Hep C Ab Screen Result",
    #                                     "Hypersegmentation Result", "HCG Result (Serum)", "Coombs Test Result",
    #                                     "HIV Result", "Urine Glucose Result(mg/dL)"])
    
    
    testNames = [x_column_name]
    testNames.extend(columnNames) #addition of time label


    ptLabs_results= df_Labs[testNames] #new data frame of just results

    #troponin contains strings which mess up plotting 
    df_Labs_results_2 = ptLabs_results
    # for col in columnNames:
    #     df_Labs_results_2[col] = ptLabs_results[col].apply(lambda x: 0 if isinstance(x, str) else x)
    df_Labs_results_2["Troponin-T"] = ptLabs_results["Troponin-T"].apply(lambda x: 0 if isinstance(x, str) else x)
    df_Labs_results_2["NT Pro BNP (B-Type Natriuretic Peptide)"] = ptLabs_results["NT Pro BNP (B-Type Natriuretic Peptide)"].apply(lambda x: 0 if isinstance(x, str) else x)
    df_Labs_results_2['Anisocytosis Result'] = df_Labs_results_2['Anisocytosis Result'].replace({'1+': 1, '2+': 2,'3+': 3})
    df_Labs_results_2['Macrocytes Result'] = df_Labs_results_2['Macrocytes Result'].replace({'1+': 1, '2+': 2,'3+': 3})

    ptLabs_results_2 = df_Labs_results_2
    dictOfFilteredLabs[ptID] = ptLabs_results_2


C:\Users\maega\AppData\Local\Temp\ipykernel_31568\1747423668.py:44: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_Labs_results_2["Troponin-T"] = ptLabs_results["Troponin-T"].apply(lambda x: 0 if isinstance(x, str) else x)
C:\Users\maega\AppData\Local\Temp\ipykernel_31568\1747423668.py:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_Labs_results_2["NT Pro BNP (B-Type Natriuretic Peptide)"] = ptLabs_results["NT Pro BNP (B-Type Natriuretic Peptide)"].apply(lambda x: 0 if isinstance(x, str) else x)
C:\U

checking to see if a patient has a lab and how many entries they have for that lab

In [39]:
dict_bool_data_collected = {}

dict_numb_data_collected = {}

for ptID in list(dictOfFilteredLabs.keys()):
    #get the patient data 
    ptLabData = dictOfFilteredLabs[ptID]

    #get number of labs collected 
    lengthOfData = len(ptLabData[x_column_name])
    if lengthOfData > 1: 
        numb_data_collected = createBinarySum(ptLabData, columnNames, x_column_name, defaxis= 0)
        bool_data_collected = numb_data_collected.copy()
        for lab in columnNames:
            if numb_data_collected[lab] > 0: 
                #data was collected return 1
                bool_data_collected[lab]= 1
            elif numb_data_collected[lab] == 0: 
                #no data collected, return 0
                bool_data_collected[lab] = 0

    else: 
        numb_data_collected = createBinary(ptLabData, columnNames, x_column_name)
        numb_data_collected= numb_data_collected.drop(columns = x_column_name) #remove time column
        numb_data_collected= numb_data_collected.squeeze() #convert to series
        bool_data_collected = numb_data_collected.copy()

    #renaming the number recorded columns columns 
    orig_columnNames = numb_data_collected.index
    new_columnNames = [str(idx) + " Number of Points" for idx in orig_columnNames]
    numb_data_collected.index = new_columnNames

    #renaming columns for boolean
    orig_columnNames_bool = bool_data_collected.index
    new_columnNames_bool = [str(idx) + " binary"  for idx in orig_columnNames_bool]
    bool_data_collected.index = new_columnNames_bool

    dict_bool_data_collected[ptID] = bool_data_collected
    dict_numb_data_collected[ptID] = numb_data_collected

#convert to dataframes
df_all_boolLabs = pd.DataFrame.from_dict(dict_bool_data_collected, orient= 'index')
df_all_numbLabs = pd.DataFrame.from_dict(dict_numb_data_collected, orient='index')

#concatinate 
df_all_labs_bin_count = pd.concat([df_all_boolLabs, df_all_numbLabs], axis= 1)


In [40]:
#splitting dataframe by amyloid status
df_AL_known_labs_bin_count = df_all_labs_bin_count.loc[listALKnown]
df_AL_Unknown_labs_bin_count = df_all_labs_bin_count.loc[listOther]
outputToExcel(df_data = df_AL_known_labs_bin_count, fileName_header= "AL-KnownPts", fileName_Common= outputSheet_common, fileName_suffix= outputSheet_Suffix,
            parent_dir= outputDir, folderName=folderNameOut, sheetName= "labs bin count")
outputToExcel(df_data = df_AL_Unknown_labs_bin_count, fileName_header= "AL-UnknownPts", fileName_Common= outputSheet_common, fileName_suffix= outputSheet_Suffix,
            parent_dir= outputDir, folderName=folderNameOut, sheetName= "labs bin count")

linking all the dataframes together

In [42]:
#for amyloid status known patients
df_ALKnown_summary = pd.concat([df_ALKnown_demographics, df_AL_known_labs_bin_count,
                                df_AL_known_tx_summary, df_AL_known_notes_bin_count], axis =1 )

outputToExcel(df_data = df_ALKnown_summary, fileName_header= "AL-KnownPts", fileName_Common= "-binnary-Counts-allLabs-tx-notes-Summary-One", fileName_suffix= outputSheet_Suffix,
        parent_dir= outputDir, folderName=folderNameOut, sheetName= "ALL")

df_ALOther_summary = pd.concat([df_ALOther_demographics, df_AL_Unknown_labs_bin_count,
                                df_AL_Unknown_tx_summary, df_AL_Unknown_notes_bin_count], axis =1 )

outputToExcel(df_data = df_ALOther_summary, fileName_header= "AL-UnknownPts", fileName_Common= "-binnary-Counts-allLabs-tx-notes-Summary-One", fileName_suffix= outputSheet_Suffix,
        parent_dir= outputDir, folderName=folderNameOut, sheetName= "ALL")